# GLLVM Simulations

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from gllvm import GLLVM, PoissonGLM, BinomialGLM, GaussianGLM


seed = 123
device = "cuda"

torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


# ----------------------------------------------------------
# 1. Generate synthetic data from the ground-truth model
# ----------------------------------------------------------

gllvm0 = GLLVM(latent_dim=1, output_dim=5)
gllvm0.add_glm(PoissonGLM, idx=[0, 1], name="Poisson1")
gllvm0.add_glm(BinomialGLM, idx=[2], name="Binomial1")
gllvm0.add_glm(GaussianGLM, idx=[3, 4], name="Gaussian1")

num_samples = 10000

with torch.no_grad():
    gllvm0.wz.mul_(1.0)
    z0 = gllvm0.sample_z(num_samples)
    y0 = gllvm0.sample(z=z0)

# ----------------------------------------------------------
# 2. Build a fresh model that will be trained with VI
# ----------------------------------------------------------

gllvm = GLLVM(latent_dim=1, output_dim=5)
gllvm.add_glm(PoissonGLM, idx=[0, 1], name="Poisson1")
gllvm.add_glm(BinomialGLM, idx=[2], name="Binomial1")
gllvm.add_glm(GaussianGLM, idx=[3, 4], name="Gaussian1")

# ----------------------------------------------------------
# 3. Build the encoder q(z|y)
# ----------------------------------------------------------

class Encoder(nn.Module):
    def __init__(self, input_dim=5, latent_dim=1, hidden=32):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, hidden),
            nn.ReLU(),
        )
        self.mean = nn.Linear(hidden, latent_dim)
        self.logvar = nn.Linear(hidden, latent_dim)

    def forward(self, y):
        h = self.net(y)
        mu = self.mean(h)
        logvar = self.logvar(h)
        return mu, logvar

encoder = Encoder(input_dim=5, latent_dim=1)
optimizer = optim.Adam(list(gllvm.parameters()) + list(encoder.parameters()), lr=1e-3)

# ----------------------------------------------------------
# 4. VI training loop: Standard VAE
# ----------------------------------------------------------

gllvm.to(device)
gllvm0.to(device)
encoder.to(device)
y0 = y0.to(device)


batch_size = 1000
num_epochs = 200

dataset = y0
n = len(dataset)

for epoch in range(num_epochs):
    perm = torch.randperm(n)

    total_elbo = 0.0
    for i in range(0, n, batch_size):
        idx = perm[i:i+batch_size].to(device)
        y = dataset[idx]

        # Encode q(z|y)
        mu, logvar = encoder(y)
        std = torch.exp(0.5 * logvar)

        # Reparameterization
        eps = torch.randn_like(std)
        z = mu + eps * std

        # Decoder log-likelihood
        logpy_z = gllvm.log_prob(y, z=z).sum(dim=-1)  # sum over response dims

        # KL(q||p)
        kl = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp(), dim=1)

        elbo = (logpy_z - kl).mean()
        loss = -elbo

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_elbo += elbo.item()

    print(f"Epoch {epoch+1}: ELBO={total_elbo:.2f}")



Epoch 1: ELBO=-86.53
Epoch 2: ELBO=-79.47
Epoch 3: ELBO=-75.08
Epoch 4: ELBO=-71.40
Epoch 5: ELBO=-68.86
Epoch 6: ELBO=-67.74
Epoch 7: ELBO=-67.50
Epoch 8: ELBO=-67.27
Epoch 9: ELBO=-67.12
Epoch 10: ELBO=-67.25
Epoch 11: ELBO=-67.10
Epoch 12: ELBO=-67.08
Epoch 13: ELBO=-66.96
Epoch 14: ELBO=-67.02
Epoch 15: ELBO=-66.75
Epoch 16: ELBO=-66.65
Epoch 17: ELBO=-66.80
Epoch 18: ELBO=-66.71
Epoch 19: ELBO=-66.71
Epoch 20: ELBO=-66.80
Epoch 21: ELBO=-66.74
Epoch 22: ELBO=-66.60
Epoch 23: ELBO=-66.66
Epoch 24: ELBO=-66.55
Epoch 25: ELBO=-66.54
Epoch 26: ELBO=-66.50
Epoch 27: ELBO=-66.48
Epoch 28: ELBO=-66.40
Epoch 29: ELBO=-66.52
Epoch 30: ELBO=-66.45
Epoch 31: ELBO=-66.59
Epoch 32: ELBO=-66.32
Epoch 33: ELBO=-66.43
Epoch 34: ELBO=-66.38
Epoch 35: ELBO=-66.54
Epoch 36: ELBO=-66.39
Epoch 37: ELBO=-66.44
Epoch 38: ELBO=-66.34
Epoch 39: ELBO=-66.34
Epoch 40: ELBO=-66.33
Epoch 41: ELBO=-66.43
Epoch 42: ELBO=-66.26
Epoch 43: ELBO=-66.30
Epoch 44: ELBO=-66.32
Epoch 45: ELBO=-66.31
Epoch 46: ELBO=-66.

In [ ]:
from gllvm.simulations import make_mixed, simulate

# define a setting *for the article*
g0 = make_mixed(
    n_latent=1,
    gaussian=2,
    poisson=2,
    binomial=1
)

# generate data
y0, z0 = simulate(g0, n_samples=20000, device="cuda")


In [2]:
with torch.no_grad():
    print(torch.mean((gllvm0.wz - gllvm.wz)**2), torch.mean((gllvm0.wz + gllvm.wz)**2))

tensor(1.3265, device='cuda:0') tensor(0.0002, device='cuda:0')


In [3]:
gllvm0.wz *-1

tensor([[ 0.1115, -0.1204,  0.3696,  0.2404,  1.1969]], device='cuda:0',
       grad_fn=<MulBackward0>)

In [4]:
gllvm.wz

Parameter containing:
tensor([[ 0.1333, -0.1149,  0.3499,  0.2473,  1.2034]], device='cuda:0',
       requires_grad=True)

In [5]:
print("True W_z:\n", gllvm0.wz)
print("Estimated W_z:\n", gllvm.wz)

print("True bias:\n", gllvm0.bias)
print("Estimated bias:\n", gllvm.bias)

print("True scale:\n", gllvm0.scale)
print("Estimated scale:\n", gllvm.scale)


True W_z:
 Parameter containing:
tensor([[-0.1115,  0.1204, -0.3696, -0.2404, -1.1969]], device='cuda:0',
       requires_grad=True)
Estimated W_z:
 Parameter containing:
tensor([[ 0.1333, -0.1149,  0.3499,  0.2473,  1.2034]], device='cuda:0',
       requires_grad=True)
True bias:
 Parameter containing:
tensor([0., 0., 0., 0., 0.], device='cuda:0', requires_grad=True)
Estimated bias:
 Parameter containing:
tensor([ 0.0024,  0.0105,  0.0041, -0.0082,  0.0113], device='cuda:0',
       requires_grad=True)
True scale:
 tensor([1., 1., 1., 1., 1.], device='cuda:0', grad_fn=<ExpBackward0>)
Estimated scale:
 tensor([1.0000, 1.0000, 1.0000, 1.0130, 0.9506], device='cuda:0',
       grad_fn=<ExpBackward0>)
